In [1]:
import pandas as pd
import plotly.graph_objects as go

In [6]:
data_path = "../data/"
df_account = pd.read_csv(data_path+'account.csv', delimiter=';')
df_card = pd.read_csv(data_path+'card.csv', delimiter=';')
df_client = pd.read_csv(data_path+'client.csv', delimiter=';',
                       dtype={'birth_number':str})
df_disp = pd.read_csv(data_path+'disp.csv', delimiter=';')
df_district = pd.read_csv(data_path+'district.csv', delimiter=';')
df_loan = pd.read_csv(data_path+'loan.csv', delimiter=';')
df_trans = pd.read_csv(data_path+'trans.csv', delimiter=';', 
                       dtype={'date':str,'bank':str})
df_order = pd.read_csv(data_path+'order.csv', delimiter=';')

In [7]:
def prepare_dfs():
    """
    general data manipulation of some tables in order to get the correct information in an appropriate format
    """
    global df_trans, df_client, df_trans, df_account, df_loan, df_order, df_disp, df_card
    
    #changes date to datetime
    df_trans['date'] = "19" + df_trans['date']
    df_trans['date'] = pd.to_datetime(df_trans['date'], format='%Y%m%d')
    
    #adds column "gender" and changes "birth_number" to datetime and generates an age column (reference day is 1998-12-31)
    df_client['gender_id'] = pd.Series(df_client['birth_number'].str[2:4], dtype=int)
    df_client['birth_number'] = df_client['birth_number'].astype('int64')
    df_client.loc[df_client.gender_id > 50, 'gender'] = "f"
    df_client.loc[df_client.gender_id < 13, 'gender'] = "m"
    df_client = df_client.drop(columns=['gender_id'])
    
    df_client.loc[df_client.gender == 'f', 'birth_number'] = df_client['birth_number'] - 5000
    df_client['birth_number'] = "19" + df_client['birth_number'].astype(str)
    df_client['birth_number'] = pd.to_datetime(df_client['birth_number'], format='%Y%m%d')
    
    df_client['age'] = ((pd.Series(['1998-12-31']*df_client.shape[0], dtype='datetime64[ns]') - df_client['birth_number']) / 365.25).dt.days
    
    #rename column names to avoid name conflicts while merging
    df_trans = df_trans.rename(columns={'date':'date_trans','type':'type_trans','amount':'amount_trans','k_symbol':'k_symbol_trans'})
    df_account = df_account.rename(columns={'date':'date_account'})
    df_loan = df_loan.rename(columns={'date':'date_loan','amount':'amount_loan'})
    df_order = df_order.rename(columns={'amount':'amount_order','k_symbol':'k_symbol_order'})
    df_disp = df_disp.rename(columns={'type':'type_disp'})
    df_card = df_card.rename(columns={'type':'type_card'})
    
prepare_dfs()

In [16]:
df_trans_merged = df_client.merge(df_disp, on='client_id').merge(df_card, on='disp_id').merge(df_account, on='account_id').merge(df_trans, on='account_id')
df_assets = df_trans_merged.sort_values(['client_id','date_trans']).groupby(by=['client_id']).tail(1)
df_assets

,client_id,birth_number,district_id_x,gender,age,disp_id,account_id,type_disp,card_id,type_card,...,date_account,trans_id,date_trans,type_trans,operation,amount_trans,balance,k_symbol_trans,bank,account
129,9,1935-10-16,60,m,63,9,7,OWNER,1,gold,...,961124,3530687,1998-12-31,PRIJEM,NaN,320.2,68495.5,UROK,NaN,NaN
253,19,1942-12-28,47,m,56,19,14,OWNER,2,classic,...,961127,3530850,1998-12-31,PRIJEM,NaN,179.2,39313.9,UROK,NaN,NaN
597,41,1968-08-27,22,m,30,41,33,OWNER,3,gold,...,930826,3531484,1998-12-31,PRIJEM,NaN,429.5,112405.5,UROK,NaN,NaN
1006,42,1935-08-17,68,m,63,42,34,OWNER,4,classic,...,930917,3531547,1998-12-31,PRIJEM,NaN,235.3,50114.2,UROK,NaN,NaN
1327,51,1979-12-02,36,f,19,51,43,OWNER,5,junior,...,940612,3531914,1998-12-31,PRIJEM,NaN,255.8,60766.5,UROK,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220743,13620,1970-07-26,16,m,28,13312,11079,OWNER,1230,classic,...,951110,3527516,1998-12-31,PRIJEM,NaN,358.7,101756.8,UROK,NaN,NaN
221068,13690,1963-05-16,70,m,35,13382,11138,OWNER,1233,classic,...,950818,3528046,1998-12-31,PRIJEM,NaN,193.3,56023.0,UROK,NaN,NaN
221220,13694,1976-01-20,1,f,22,13386,11141,OWNER,1234,classic,...,970211,3528068,1998-12-31,PRIJEM,NaN,461.8,92960.0,UROK,NaN,NaN
221559,13750,1980-11-13,12,f,18,13442,11186,OWNER,1239,junior,...,941124,3528438,1998-12-31,PRIJEM,NaN,358.3,84779.6,UROK,NaN,NaN


In [19]:
fig_age_balance = go.Figure(data=go.Scatter(x=df_assets['age'], y=df_assets['balance'], mode='markers'))
fig_age_balance

In [20]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species",
                 size='petal_length', hover_data=['petal_width'])
fig.show()